In [53]:
import daria as da
import cv2
import numpy as np
#import greyscale image
# im = cv2.imread("../images/originals/Baseline.jpg", 0)

# img = da.Image(im, [0,0], 280, 150)
# img.show()

# img.coarsen(0.001,0.001)
# img.show()



def backward_diff_x(im):
    img = im.astype(float)
    return np.c_[img,img[:,-1]][:,1:]-img

def forward_diff_x(im):
    img = im.astype(float)
    return img - np.c_[img[:,0],img][:,:-1]

def laplace_x(im):
    return 0.5*(forward_diff_x(backward_diff_x(im))+backward_diff_x(forward_diff_x(im)))

def backward_diff_y(im):
    img = im.astype(float)
    return np.r_[img,[img[-1,:]]][1:,:]-img

def forward_diff_y(im):
    img = im.astype(float)
    return img - np.r_[[img[0,:]],img][:-1,:]

def laplace_y(im):
    return 0.5*(forward_diff_y(backward_diff_y(im))+backward_diff_y(forward_diff_y(im)))

def laplace(im):
    return laplace_x(im)+laplace_y(im)


from math import sqrt

def norm(x):
    return sqrt(np.tensordot(x,x))

def richardson(operator, b, im0, tol, omega, max_iter= 1000):
    res = b-operator(im0)
    im = im0
    nr = norm(res)
    iteration = 0
    while nr>tol and iteration<max_iter:
        im = im + omega*res
        res = b - operator(im)
        nr = norm(res)
        print(nr)
        iteration += 1
    print(iteration)

    return im


def op(im):
    return im - 0.1*laplace(im)


def restriction(im, diff = 0.5):
    return cv2.resize(im, (0,0), fx = diff, fy = diff)

def prolong(im, dim):
    #return cv2.resize(im, (0,0), fx = diff, fy = diff)
    return cv2.resize(im, dim)

def multigrid_richardson(op, b, im0, tol, rich_param, max_iter = 100, depth = 3):
    
    # pre-smoothing 
    im = richardson(op, b, im0, tol=0.1, omega = rich_param, max_iter = 1)
    
    residual = b-op(im)

    # restriction
    rhs = restriction(residual)

    # error-correction
    eps = richardson(op, rhs, rhs, tol = 0.01, omega=rich_param, max_iter = 1000)

    # prolongation
    im += prolong(eps, (residual.shape[1],residual.shape[0]))


    # post-smoothing
    im = richardson(op, b, im, tol=0.1, omega = rich_param, max_iter = 1)


    return im


def multigrid_richardson(op, b, im0, tol, rich_param, max_iter = 100, depth = 3):
    
    # pre-smoothing 
    im = richardson(op, b, im0, tol=0.1, omega = rich_param, max_iter = 1)
    
    residual = b-op(im)

    # restriction
    rhs = restriction(residual)

    # error-correction
    eps = richardson(op, rhs, rhs, tol = 0.01, omega=rich_param, max_iter = 1000)

    # prolongation
    im += prolong(eps, (residual.shape[1],residual.shape[0]))


    # post-smoothing
    im = richardson(op, b, im, tol=0.1, omega = rich_param, max_iter = 1)


    return im

def im_product(im1, im2):
    return np.tensordot(im1,im2)

def cg(op, b, im0, tol, max_iter = 1000):
    im = im0
    res = op(im0)-b
    p = -res
    for _ in range(max_iter):
        po = op(p)
        alpha = -im_product(res,p)/im_product(po,p)
        im = im + alpha*p
        res = op(im)-b
        beta = im_product(res,po)/im_product(po,p)
        p = -res + beta*p
        print(norm(res))
        if norm(res)<tol:
            break
    alpha = -im_product(res,p)/im_product(op(p),p)
    im = im + alpha*p
    return im



In [52]:
testim = cv2.imread("../images/originals/Baseline.jpg", 0).astype(float)
testim = cv2.resize(testim, (0,0), fx = 1, fy = 1)
regularized_im = cg(op, testim , testim, 0.01)
im_r = regularized_im.astype(np.uint8)
cv2.imshow("v",im_r)
cv2.waitKey(0)
cv2.destroyAllWindows()

826.8231936624679
123.48778492163741
17.640694225156235
2.510667982073799
0.3629437137038764
0.05229716285036945
0.007564231139858658


In [55]:
def shrink(x,l, norm):
    n = norm(x)
    return x/n*max(n-l,0)


def split_bregman(rhs, linsolv, diffx = backward_diff_x, diffy = backward_diff_y, l = 1):
    im = rhs
    dx = np.zeros(rhs.shape)
    dy = np.zeros(rhs.shape)
    bx = np.zeros(rhs.shape)
    by = np.zeros(rhs.shape)
    run = True
    while run:
        im = linsolve(lhsoperator, rhsoperator(rhs,dx,dy,bx,by), im, 0.1, 10)
        dx = shrink(diffx(im)+bx, 1/l)
        dy = shrink(diffy(im)+by, 1/l)
        bx = bx + diffx(im)-dx
        by = by + diffy(im)-dy
    return im




#for solving the problem min_u |dxu|+|dyu|+mu/2|u-f|^2_2, with penalization parameter l
def lhsoperator(im, mu = 1, l = 1):
    return mu*im - l*laplace(im)

def rhsoperator(rhs, dx, dy, bx, by, mu, l, diffx = forward_diff_x, diffy = forward_diff_y):
    return mu*rhs+l*(diffx(dx-bx)+diffy(dy-by))






<class 'function'>


In [59]:
d= np.zeros(testim.shape)

ValueError: too many values to unpack (expected 2)